## Yellow Taxi Tripdata
### Analyse der New Yorker Yellow Taxi Tripdata
In diesem Projekt tauchen wir in die faszinierende Welt der New Yorker Taxis ein. Der Datensatz enthält Millionen von Fahrten der gelben Taxis, inklusive genauer Informationen zu jeder Tour: Wann und wo die Fahrt begann und endete, wie lange sie dauerte, wie viele Passagiere mitfuhren und wie hoch der Fahrpreis war.

Unser Ziel ist es, diese umfangreichen Daten zu analysieren, um spannende Muster und Erkenntnisse zu entdecken. Wir werden zum Beispiel herausfinden, zu welchen Zeiten die meisten Taxis unterwegs sind, welche Strecken am beliebtesten sind oder wie sich die Fahrpreise zusammensetzen.

Durch die Arbeit mit diesem realen Datensatz lernen wir nicht nur, wie man große Datenmengen verarbeitet, sondern erhalten auch einen tiefen Einblick in das pulsierende Verkehrsnetz von New York City.

In [1]:
# import der
import pandas as pd
import glob
#import pyarrow
from fastparquet import ParquetFile
#from fastparquet import write

### Data Preperation
#### Warum CSV für diesen Datensatz ungeeignet ist
Dieser Datensatz ist mit bis zu 16 GB sehr groß. Das CSV-Format ist für diese Menge an Daten nicht ideal. Da es zeilenorientiert ist und die Komprimierung ineffizient ist, dauert das Einlesen und Speichern viel zu lange. Diese Zeit können wir uns sparen.

#### Warum Parquet die bessere Wahl ist
Das Parquet-Format ist die perfekte Lösung für solch große Datensätze. Es ist ein modernes, spalten-orientiertes Datenformat. Das bedeutet, dass beim Einlesen nur die Spalten geladen werden, die du wirklich brauchst – das spart enorm viel Zeit und Arbeitsspeicher. Parquet-Dateien sind außerdem hoch komprimiert, was die Dateigröße drastisch reduziert (in unserem Fall von 16 GB auf ca. 3 GB).

Parquet ist daher der Standard für Big-Data-Anwendungen und wird auch in Data-Warehouse-Systemen wie AWS Redshift häufig verwendet. Da auch Pandas sehr gut mit Parquet-Dateien umgehen kann, ist das die ideale Arbeitsgrundlage für uns.

#### Unser Plan
Wir werden die Daten aus den einzelnen CSV-Dateien zunächst mit Pandas einlesen und dann direkt in ein einziges, großes Parquet-File umwandeln. Dieses File wird unsere neue, deutlich effizientere Arbeitsgrundlage für alle weiteren Schritte sein.

In [ ]:
#### Download des Dataserts und entpacken der Dateien


In [2]:
# Das Muster, das zu allen 12 Dateien passt
dateipfad_muster = 'NYTaxi-TripData/yellow_tripdata_*.csv'
dateiliste = glob.glob(dateipfad_muster)

# Erstelle eine leere Liste, um die DataFrames zu speichern
dfs = []

# Iteriere durch die Liste der Dateien und lade jede einzelne in eine DataFrame
for datei in dateiliste:
    #df = pd.read_csv(datei) <- erzeugt Warnungen
    # pd.read_csv(datei, low_memory=False) <- vermeidet die Warnungen führt aber zu hohem Speicherverbrauch
    df = pd.read_csv(datei, dtype={6: str}) # legt die problematische Spalte auf den Datentyp string fest
    dfs.append(df)

# Verbinde alle DataFrames zu einer einzigen, großen DataFrame
yellow_trips_df = pd.concat(dfs, ignore_index=True)

In [3]:
# Ausgeben des Head-Bereichs
yellow_trips_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-11-01 00:30:41,2019-11-01 00:32:25,1.0,0.0,1.0,N,145,145,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1.0,2019-11-01 00:34:01,2019-11-01 00:34:09,1.0,0.0,1.0,N,145,145,2.0,2.5,0.5,0.5,0.00,0.0,0.3,3.80,0.0
2,2.0,2019-11-01 00:41:59,2019-11-01 00:42:23,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
3,2.0,2019-11-01 00:02:39,2019-11-01 00:02:51,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
4,2.0,2019-11-01 00:18:30,2019-11-01 00:18:39,2.0,0.0,1.0,N,226,226,2.0,2.5,0.0,0.5,0.00,0.0,0.3,3.30,0.0


In [4]:
# Ein erster Überblick über die vorhandenen Datentypen
# was fällt dir auf?
yellow_trips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101246797 entries, 0 to 101246796
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 13.6+ GB


In [5]:
# Wir wandeln den Dataframe in ein Parquet-File um
yellow_trips_df.to_parquet('NYTaxi-TripData/yellow_tripdata.parquet', engine='fastparquet')

In [6]:
# Einlesen des Parquet-Files in ein Dataframe
pf = ParquetFile('NYTaxi-TripData/yellow_tripdata.parquet')
df = pf.to_pandas()
# Beachte die verkürzte Zeit beim Einlesen. Ca 2,5 Minuten bei CSV-Files und ca 0,5 Minuten bei Parquet-File.
# (Abhängig von der Leistungsfähigkeit des Computers)

In [7]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-11-01 00:30:41,2019-11-01 00:32:25,1.0,0.0,1.0,N,145,145,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1.0,2019-11-01 00:34:01,2019-11-01 00:34:09,1.0,0.0,1.0,N,145,145,2.0,2.5,0.5,0.5,0.00,0.0,0.3,3.80,0.0
2,2.0,2019-11-01 00:41:59,2019-11-01 00:42:23,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
3,2.0,2019-11-01 00:02:39,2019-11-01 00:02:51,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
4,2.0,2019-11-01 00:18:30,2019-11-01 00:18:39,2.0,0.0,1.0,N,226,226,2.0,2.5,0.0,0.5,0.00,0.0,0.3,3.30,0.0


### Fast fertig...
#### Unser Daten-Setup und Datensicherheit
Wir haben jetzt ein robustes Setup: Die originalen CSV-Dateien dienen als Backup, und die neu erstellte Parquet-Datei ist unsere effiziente Arbeitsgrundlage. Bevor wir mit der Datenbereinigung und -analyse beginnen, erstellen wir von unserem DataFrame eine Arbeitskopie. Das ist ein wichtiger Schritt, da Pandas sehr flexibel ist und unbeabsichtigte Änderungen den Datensatz schnell beschädigen könnten. So bleibt die Parquet-Datei immer intakt.

#### Einbinden der Lookup-Tabelle
Zusätzlich benötigen wir die Lookup-Tabelle, die wir ebenfalls in einen DataFrame einlesen. Da diese Datei wesentlich kleiner ist, muss sie nicht in eine Parquet-Datei umgewandelt werden. Diese Tabelle ist im Prinzip eine Referenz, die unsere Haupttabelle ergänzt. Sie verbindet beispielsweise eine `LocationID` aus den Trip-Daten mit dem passenden Zonennamen.

In [8]:
# Das werden wir später noch genauer unter die Lupe nehmen.
lookup_df = pd.read_csv('NYTaxi-TripData/taxi+_zone_lookup.csv')
lookup_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [9]:
# Erstellen der Arbeitskope der Tripdaten
ytdf = df.copy()
ytdf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-11-01 00:30:41,2019-11-01 00:32:25,1.0,0.0,1.0,N,145,145,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1.0,2019-11-01 00:34:01,2019-11-01 00:34:09,1.0,0.0,1.0,N,145,145,2.0,2.5,0.5,0.5,0.00,0.0,0.3,3.80,0.0
2,2.0,2019-11-01 00:41:59,2019-11-01 00:42:23,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
3,2.0,2019-11-01 00:02:39,2019-11-01 00:02:51,1.0,0.0,1.0,N,193,193,1.0,2.5,0.5,0.5,0.95,0.0,0.3,4.75,0.0
4,2.0,2019-11-01 00:18:30,2019-11-01 00:18:39,2.0,0.0,1.0,N,226,226,2.0,2.5,0.0,0.5,0.00,0.0,0.3,3.30,0.0


#### Wozu eine Arbeitskopie?
Ganz einfach: Zum Testen. Vielleicht möchtest du fehlende Werte ersetzten oder eine Spalte hinzufügen oder, oder, oder...

Dafür sind temporäre Kopie sinnvoll, du kannst jederzeit zu einem "originalen" Dataframe zurückwechseln. Wir brauchen das in diesem Notebook noch nicht.

### Abschluss und Ausblick
Nachdem wir nun die Vorbereitung abgeschlossen und unser Daten-Setup perfektioniert haben, kann der Spaß beginnen.

Hier ist unser Fahrplan für die nächsten Schritte:

* Phase 1: Bereinigung & Vorbereitung
Zuerst widmen wir uns der Bereinigung des großen DataFrames. Wir korrigieren die Datentypen der Spalten, insbesondere der Datums- und Zeitangaben, kümmern uns um fehlende Werte und filtern offensichtliche Fehler wie Fahrten mit 0 Dollar Preis oder 0 Minuten Dauer heraus. Ziel ist ein sauberer, zuverlässiger Datensatz.

* Phase 2: Daten verknüpfen
Sobald der große Trip-Datensatz sauber ist, verknüpfen wir ihn mit der kleinen Lookup-Tabelle. So können wir die numerischen LocationIDs den echten Zonennamen zuordnen. Das macht die Daten verständlicher und für die Analyse nutzbar.

* Phase 3: Analyse & Visualisierung
Mit den bereinigten und verknüpften Daten können wir endlich tiefer eintauchen. Wir werden die Daten analysieren, um Muster zu finden. Zum Beispiel:

Wann sind die Stoßzeiten in New York?

Welche Stadtteile sind die beliebtesten Abholorte?

Wie lang sind die Fahrten durchschnittlich und wie hoch sind die Trinkgelder?

Mit diesen Erkenntnissen können wir dann beginnen, die Daten zu visualisieren und vielleicht sogar die Zonen auf einer Karte darzustellen.

Bereit für den Start?